In [3]:
import json
import os
from PIL import Image

# Đường dẫn
json_path = "/mnt/VLAI_data/ViVQA-X/ViVQA-X_val.json"
coco_img_dir = "/mnt/VLAI_data/COCO_Images/val2014/"

# Đọc file JSON
with open(json_path, "r", encoding="utf-8") as f:
    data = json.load(f)

samples = []
for item in data:
    img_path = os.path.join(coco_img_dir, item["image_name"])
    # Mở ảnh thành PIL Image
    image = Image.open(img_path).convert("RGB")
    sample = {
        "question": item["question"],
        "image": image,  # <-- PIL Image object
        "image_path": img_path,
        "explanation": item["explanation"],  # list
        "answer": item["answer"],
        "question_id": item["question_id"]
    }
    samples.append(sample)

# Kiểm tra 1 sample
print(samples[0])

{'question': 'Đây có phải là bức ảnh chụp nhiều độ phơi sáng của vận động viên trượt tuyết mặc áo đen không?', 'image': <PIL.Image.Image image mode=RGB size=640x480 at 0x700EA1B17160>, 'image_path': '/mnt/VLAI_data/COCO_Images/val2014/COCO_val2014_000000393271.jpg', 'explanation': ['hình người mặc cùng một bộ quần áo khi trượt xuống dốc', 'có vẻ như là cùng một người trượt tuyết làm những pha nhào lộn khác nhau', 'cùng một người trượt tuyết xuất hiện nhiều lần'], 'answer': 'có', 'question_id': '393271001'}


In [4]:
from transformers import AutoModel
from transformers import AutoTokenizer
import sys
import os

# Thêm đường dẫn tới thư mục chứa processor.py
# Đường dẫn này tương đối từ vị trí của notebook
processor_path = os.path.abspath('../ViVQA/beit3/HCMUS')
if processor_path not in sys.path:
    sys.path.append(processor_path)

from processor import Processor
from PIL import Image
import torch
from IPython.display import display
from PIL import Image

device = "cuda" if torch.cuda.is_available() else "cpu"

model = AutoModel.from_pretrained("ngocson2002/vivqa-model", trust_remote_code=True).to(device)
processor = Processor()

image = Image.open(samples[0]["image_path"]).convert("RGB")
display(image)
question = samples[0]["question"]

inputs = processor(image, question, return_tensors='pt')
inputs["image"] = inputs["image"].unsqueeze(0)

model.eval()
with torch.no_grad():
    output = model(**inputs)
    logits = output.logits
    idx = logits.argmax(-1).item()

print("Predicted answer:", model.config.id2label[idx]) 


KeyboardInterrupt: 